In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline, AutoTokenizer

In [2]:
import os
os.environ['HF_HOME'] = '../../hf_home/'

In [3]:
import json
import pprint as pp
with open("../error_types.json") as f:
    err_definitions = json.load(f)
    
err_def_str = pp.pformat(err_definitions)


In [5]:
with open("../../data/real/clean/qalb-14/jsonl/dev.jsonl") as f:
    q14 = [json.loads(l) for l in f]
with open("../../data/real/clean/qalb-14+qalb-15/jsonl/dev.jsonl") as f:
    q15l1 = [json.loads(l) for l in f]
with open("../../data/real/clean/qalb-14+qalb-15/jsonl/dev-l2.jsonl") as f:
    q15l2 = [json.loads(l) for l in f]
with open("../../data/real/clean/qalb-14+qalb-15+ZAEBUC/jsonl/dev.jsonl") as f:
    zaebuc = [json.loads(l) for l in f]

In [6]:
with open("../../data/real/clean/qalb-14/jsonl/test.jsonl") as f:
    q14_test = [json.loads(l) for l in f]
with open("../../data/real/clean/qalb-14+qalb-15/jsonl/test-l1.jsonl") as f:
    q15l1_test = [json.loads(l) for l in f]
with open("../../data/real/clean/qalb-14+qalb-15/jsonl/test-l2.jsonl") as f:
    q15l2_test = [json.loads(l) for l in f]
with open("../../data/real/clean/qalb-14+qalb-15+ZAEBUC/jsonl/test.jsonl") as f:
    zaebuc_test = [json.loads(l) for l in f]

In [ ]:

import torch
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
tokenizer.pad_token_id = tokenizer.eos_token_id
pipe = pipeline("text-generation",
                model="meta-llama/Meta-Llama-3.1-8B-Instruct",
                tokenizer=tokenizer,
                device_map="cuda",
                trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [16]:
messages = [
    {"role": "system", "content": "You are an AI assistant that is an expert in the Arabic language. Your Goal is to correct the grammatical mistakes in any given "
                                  "sentence. You should apply minimal changes to output the same sentence but free of any grammatical errors."
                                  "Here are the definition of common grammatical errors in arabic and examples on how to fix them:"
                                  f"\n{err_def_str}\n"},
    {"role": "user", "content": f"{q15l1[1]['source']}"}, {"role": "assistant", "content": f"{q15l1[1]['correct']}"},
    {"role": "user", "content": f"{q15l1[2]['source']}"}, {"role": "assistant", "content": f"{q15l1[2]['correct']}"},
    # {"role": "user", "content": f"{q15l1[4]['source']}"}, {"role": "assistant", "content": f"{q15l1[4]['correct']}"},
    # {"role": "user", "content": f"{q15l1[8]['source']}"}, {"role": "assistant", "content": f"{q15l1[8]['correct']}"},
    {"role": "user", "content": f"{q15l1_test[0]['source']}"},
]
pipe(messages, max_new_tokens=2048)

[{'generated_text': [{'role': 'system',
    'content': "You are an AI assistant that is an expert in the Arabic language. Your Goal is to correct the grammatical mistakes in any given sentence. You should apply minimal changes to output the same sentence but free of any grammatical errors.Here are the definition of common grammatical errors in arabic and examples on how to fix them:\n{'MG': {'description': 'Words are Merged',\n        'example': {'correct': 'ذهبت البارحة', 'incorrect': 'ذهبتالبارحة'}},\n 'MI': {'description': 'Incorrect Word Inflection',\n        'example': {'correct': 'عارف', 'incorrect': 'معروف'}},\n 'MT': {'description': 'Incorrect Verb Tense',\n        'example': {'correct': 'أفرحتني', 'incorrect': 'تفرحني'}},\n 'OA': {'description': 'Yaa / Alif Maksura Confusion',\n        'example': {'correct': 'على', 'incorrect': 'علي'}},\n 'OC': {'description': 'Wrong Order of Letters',\n        'example': {'correct': 'تربينا', 'incorrect': 'تبرينا'}},\n 'OD': {'description': '

In [12]:
messages = [
    {"role": "system", "content": "You are an AI assistant that is an expert in the Arabic language. Your Goal is to correct the grammatical mistake of the given type in any given "
                                  "word or phrase."},
    {"role": "user", "content": f"error type: XG\nفرحوا"}, {"role": "assistant", "content": f"فرحن"},
    {"role": "user", "content": f"افرغت"},
]
pipe(messages, max_length=2048)

[{'generated_text': [{'role': 'system',
    'content': 'You are an AI assistant that is an expert in the Arabic language. Your Goal is to correct the grammatical mistake of the given type in any given word or phrase.'},
   {'role': 'user', 'content': 'error type: XG\nفرحوا'},
   {'role': 'assistant', 'content': 'فرحن'},
   {'role': 'user', 'content': 'افرغت'},
   {'role': 'assistant',
    'content': 'الكلمة "فرحوا" مكتوبة بشكل صحيح. ولكن الكلمة "افرغت" تحتاج إلى تصحيح. في اللغة العربية، الفعل "أفرغت" يعني أن شيئا ما كان يحتوي على شيء آخر ولكن الآن هذا الشيء الآخر غير موجود. لذا، الكلمة الصحيحة هي "أفرغت" وليس "فرغت".'}]}]

In [17]:
messages = [
    {"role": "system", "content": "You are an AI assistant fluent in Arabic. Your Goal is to correct the grammatical mistakes in any given "
                                  "sentence. You shold only output the version of the sentence free of any grammatical errors without "
                                  "adding anything else or changing the formating."
                                  "Here are the definition of common grammatical errors in arabic and examples on how to fix them:"
                                  f"\n{err_def_str}\n"},
    {"role": "user", "content": f"{q14[2]['source']}"}, {"role": "assistant", "content": f"{q14[2]['correct']}"},
    {"role": "user", "content": f"{q15l1[32]['source']}"}, {"role": "assistant", "content": f"{q15l1[32]['correct']}"},
    {"role": "user", "content": f"{q15l2[64]['source']}"}, {"role": "assistant", "content": f"{q15l2[64]['correct']}"},
    {"role": "user", "content": f"{zaebuc[1]['source']}"}, {"role": "assistant", "content": f"{zaebuc[1]['correct']}"},
    {"role": "user", "content": f"{q15l2_test[6]['source']}"},
]
pipe(messages, max_length=8192)

In [ ]:
messages = [
    {"role": "system", "content": "You are an AI assistant fluent in Arabic. Your Goal is to correct the grammatical mistakes in any given "
                                  "sentence. You shold only output the version of the sentence free of any grammatical errors without "
                                  "adding anything else or changing the formating."
                                  "Here are the definition of common grammatical errors in arabic and examples on how to fix them:"
                                  f"\n{err_def_str}\n"},
    {"role": "user", "content": f"{q14[2]['source']}"}, {"role": "assistant", "content": f"{q14[2]['correct']}"},
    {"role": "user", "content": f"{q15l1[32]['source']}"}, {"role": "assistant", "content": f"{q15l1[32]['correct']}"},
    {"role": "user", "content": f"{q15l2[64]['source']}"}, {"role": "assistant", "content": f"{q15l2[64]['correct']}"},
    {"role": "user", "content": f"{zaebuc[1]['source']}"}, {"role": "assistant", "content": f"{zaebuc[1]['correct']}"},
    {"role": "user", "content": f"{q15l1_test[0]['source']}"},
]
pipe(messages, max_length=8192)

In [ ]:
messages = [
    {"role": "system", "content": "You are an AI assistant fluent in Arabic. Your Goal is to correct the grammatical mistakes in any given "
                                  "sentence. You shold only output the version of the sentence free of any grammatical errors without "
                                  "adding anything else or changing the formating."
                                  "Here are the definition of common grammatical errors in arabic and examples on how to fix them:"
                                  f"\n{err_def_str}\n"},
    {"role": "user", "content": f"{q14[2]['source']}"}, {"role": "assistant", "content": f"{q14[2]['correct']}"},
    {"role": "user", "content": f"{q15l1[32]['source']}"}, {"role": "assistant", "content": f"{q15l1[32]['correct']}"},
    {"role": "user", "content": f"{q15l2[64]['source']}"}, {"role": "assistant", "content": f"{q15l2[64]['correct']}"},
    {"role": "user", "content": f"{zaebuc[1]['source']}"}, {"role": "assistant", "content": f"{zaebuc[1]['correct']}"},
    {"role": "user", "content": f"{q15l1_test[500]['source']}"},
]
pipe(messages, max_length=8192)

In [ ]:
messages = [
    {"role": "system", "content": "You are an AI assistant fluent in Arabic. Your Goal is to correct the grammatical mistakes in any given "
                                  "sentence. You shold only output the version of the sentence free of any grammatical errors without "
                                  "adding anything else or changing the formating."
                                  "Here are the definition of common grammatical errors in arabic and examples on how to fix them:"
                                  f"\n{err_def_str}\n"},
    {"role": "user", "content": f"{q14[2]['source']}"}, {"role": "assistant", "content": f"{q14[2]['correct']}"},
    {"role": "user", "content": f"{q15l1[32]['source']}"}, {"role": "assistant", "content": f"{q15l1[32]['correct']}"},
    {"role": "user", "content": f"{q15l2[64]['source']}"}, {"role": "assistant", "content": f"{q15l2[64]['correct']}"},
    {"role": "user", "content": f"{zaebuc[1]['source']}"}, {"role": "assistant", "content": f"{zaebuc[1]['correct']}"},
    {"role": "user", "content": f"{zaebuc_test[16]['source']}"},
]
pipe(messages, max_length=8192)

In [29]:
with open('../../data/real/clean/qalb-14+qalb-15+ZAEBUC/annotations/qalb-14+qalb-15+ZAEBUC_error-distribution.json') as f:
    err_dist = json.load(f)
    
pp.pprint(err_dist)
    
vals = list(err_dist.values())
import torch
vals = torch.tensor(vals)
T = 20.0
vals = torch.softmax(vals / T, dim=0) * 100
vals = vals.tolist()
for k in err_dist.keys():
    err_dist[k] = vals.pop(0)
pp.pprint(err_dist)

with open('../../data/real/clean/qalb-14+qalb-15+ZAEBUC/annotations/qalb-14+qalb-15+ZAEBUC_error-distribution_temp_20.json', 'w') as f:
    json.dump(err_dist, f, indent=2)



{'MG': 0.8653902947355987,
 'MI': 0.5618792875043245,
 'MT': 0.022360416156877307,
 'OA': 0.7504661935821386,
 'OC': 0.046324031962738266,
 'OD': 0.9527224861407609,
 'OG': 0.17837705568165518,
 'OH': 8.319846767865128,
 'OM': 0.5779956629230549,
 'ON': 0.00025313678668162983,
 'OR': 1.1644292187354974,
 'OS': 8.437892889387661e-05,
 'OT': 1.4568022073527798,
 'OW': 0.029785761899538448,
 'PC': 1.0523740011644291,
 'PM': 0.5340342409693452,
 'PT': 8.60850708361108,
 'SF': 0.136778243736974,
 'SP': 0.7119894020065309,
 'SW': 0.5003670483406883,
 'UC': 71.41570966898146,
 'XC': 0.6641465493237029,
 'XF': 0.15829487060491254,
 'XG': 0.1425160109017576,
 'XM': 0.7512256039421836,
 'XN': 0.1850429910642714,
 'XT': 0.21229738509699358}
{'MG': 1.652418851852417,
 'MI': 1.62753164768219,
 'MT': 1.584214210510254,
 'OA': 1.6429506540298462,
 'OC': 1.586113691329956,
 'OD': 1.6596496105194092,
 'OG': 1.5966206789016724,
 'OH': 2.3987860679626465,
 'OM': 1.6288433074951172,
 'ON': 1.5824639797210

In [ ]:
import json
with open("../../data/synthetic/qalb15/train_llm_rulebased_hybrid.jsonl") as f:
    syn = [json.loads(l) for l in f]

import pprint as pp

pp.pprint(syn)